## Problem Formulation

- The start and goal states are represented as co-ordinates in a 2-D space.
- Since every obstacle in the path is a polygon, it can be represented as a set of vertices as co-ordinates.
- The outline of each obstacle is represented by a pair of vertices, to represent each edge of the polygon. 
- The Euclidean distance between each point, is considered as the distance.
- The robot can move from one point to another, provided that there is a straight line segment between the two points, that does not intersect with any other object, at the cost of the distance between them. 

In [1]:
import numpy as np
import pandas as pd
import matplotlib as plt
import random
import math

In [2]:
class Obstacle():
    def __init__(self, no_vertices, vertices, edges):
        self.no_vertices = no_vertices
        self.vertices = [i for i in vertices]
        self.edges = [i for i in edges]

In [3]:
source = (0, 0)
goal = (50, 50)
obstacle_list = []

In [4]:
def create_obstacles():
    for i in range(5):
        for j in range(5):
            vertices = []
            edges = []
            sides = random.randint(3,6)
            for k in range(sides):
                x,y = random.randint((i*10)+1, (i+1)*10 - 2), random.randint((j*10)+1, (j+1)*10 - 2)
                vertices.append((x,y))
            for i in range(len(vertices)-1):
                edges.append((vertices[i], vertices[i+1]))
            edges.append((vertices[-1], vertices[0]))
            obstacle_list.append(Obstacle(sides, vertices, edges))

In [5]:
create_obstacles()

In [6]:
vertex_points = [j for i in obstacle_list for j in i.vertices]
vertex_points.append(goal)
edge_lines = [j for i in obstacle_list for j in i.edges]

In [7]:
def show_layout():
    for i in range(0, 51):
        if i % 10 == 0 and i != 50:
            print("-"*101)
        for j in range(0, 51):
            if (i,j) == source:
                print("S", end = " ")
            elif (i,j) == goal:
                print("G", end = " ")
            elif j%10 == 0:
                print("|", end = " ")
            elif (i,j) in vertex_points:
                print(".", end = " ")
            else:
                print(" ", end = " ")
        print()
        if i == 50:
            print("-"*101)

In [8]:
show_layout()

-----------------------------------------------------------------------------------------------------
S                   |                   |                   |                   |                   | 
|                   |                   |                   |                   |                   | 
|                   |                   |                   |                   |                   | 
|   .           .   |                   |                   |                   |                   | 
|                   |                   |                   |                   |                   | 
|             .     |                   |                   |                   |                   | 
|                   |                   |                   |                   |                   | 
|               .   |                   |                   |                   |                   | 
|   . .             |                   |                   |             

### There are two possibilities where two line segments A, B do not intersect:
* End points of A lie on the same side of line extended from B.
* End points of B lie on the same side of line extended from A.

In [9]:
def check_intersection(new_line, existing_line):
    #Check if endpoints of new line lie on same side of existing line 
    x1, y1, x2, y2 = existing_line[0][0], existing_line[0][1], existing_line[1][0], existing_line[1][1]
    if x2!=x1:
        m = (y2 - y1)/(x2 - x1) 
    #Eqn is (y-y1) = m*(x-x1) => m*(x-x1) - (y-y1) = 0
    #Substituting values for x, y
    x, y = new_line[0][0], new_line[0][1]
    if x1==x2:
        r1 = True if (x-x1)>0 else False
    else:
        r1 = True if (m*(x-x1) - (y-y1))>0 else False
    x, y = new_line[1][0], new_line[1][1]
    if x1==x2:
        r2 = True if (x-x1)>0 else False
    else:
        r2 = True if (m*(x-x1) - (y-y1))>0 else False
    result = (r1 == r2)
    if result:
        #Check if endpoints of existing line lie on same side of new line
        x1, y1, x2, y2 = new_line[0][0], new_line[0][1], new_line[1][0], new_line[1][1]
        if x2!=x1:
            m = (y2 - y1)/(x2 - x1)
        #Eqn is (y-y1) = m*(x-x1) => m*(x-x1) - (y-y1) = 0
        #Substituting values for x, y
        x, y = existing_line[0][0], existing_line[0][1]
        if x1==x2:
            r1 = True if (x-x1)>0 else False
        else:
            r1 = True if (m*(x-x1) - (y-y1))>0 else False
        x, y = existing_line[1][0], existing_line[1][1]
        if x1==x2:
            r2 = True if (x-x1)>0 else False
        else:
            r2 = True if (m*(x-x1) - (y-y1))>0 else False
        result = (r1 == r2)
    return result

In [10]:
#Heuristic is SLD from point to goal
def compute_heuristic(dist_travelled, point, goal):
    g_n = dist_travelled
    h_n = math.sqrt((goal[0]-point[0])**2 + (goal[1]-point[1])**2) 
    f_n = g_n + h_n
    return f_n

In [20]:
def successor_state(point, dist_travelled, goal):
    min_vertex = point
    min_cost = compute_heuristic(dist_travelled, point, goal)
    for vertex in vertex_points:
        flag = False
        new_line = (point, vertex)
        for edge in edge_lines:
            if check_intersection(new_line, edge):
                flag = True
                break
        if flag:
            continue
        dist = math.sqrt((vertex[0]-point[0])**2 + (vertex[1]-point[1])**2) 
        cost = compute_heuristic(dist_travelled+dist, vertex, goal)
        print(vertex, dist, cost)
        if cost<min_cost:
            min_vertex = vertex
            min_cost = cost
    return min_vertex, min_cost

In [23]:
def route_find(start, end):
    node = start
    dist = 0
    route = [node]
    while node != end:
        node, dist = successor_state(node, dist, end)
        #print(node)
        route.append(node)
    return route

In [24]:
route_find(source, goal)

KeyboardInterrupt: 